In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import glob
from datetime import datetime
from IPython.display import clear_output

!pip freeze > requirements.txt


In [2]:
def app_log(nv,msg):
    print(" "*nv + "[{}] INFO - {}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"),msg))
    

In [3]:

app_log(0,"Limpando ambiente")

if not os.path.exists("database\db"):
    os.makedirs("database\db")

if os.path.exists("database\db\dbo.db"):
  os.remove("database\db\dbo.db")

app_log(0,"Criando banco de dados")
conn = sqlite3.connect("database\db\dbo.db")
conn.execute("ATTACH DATABASE 'database\db\dbo.db' AS 'dbo';")
cur  = conn.cursor()

app_log(0,"Criando tabelas no banco de dados")
sql_file = open(r"database\ddl\create_tables.sql",encoding="ISO-8859-1")
sql_as_string = sql_file.read()
sql_file.close()
cur.executescript(sql_as_string)

conn.commit()
conn.close()


[22/01/2024 23:48:42] INFO - Limpando ambiente
[22/01/2024 23:48:42] INFO - Criando banco de dados
[22/01/2024 23:48:42] INFO - Criando tabelas no banco de dados


In [4]:
conn = sqlite3.connect("database\db\dbo.db")
dir_list = glob.glob('data/*.csv')
app_log(0,"Carga de dados no banco de dados")
app_log(1,"Tabelas Dimensao ")
for arq in filter(lambda file: "Dim" in file,dir_list):
    cursor = conn.execute("select * from {}".format(arq[5:-4]))
    colunas = [description[0] for description in cursor.description]
    columnsQuery = cursor.execute(f"pragma table_info('{arq[5:-4]}')")
    columnInfos = cursor.fetchall()
    columnsNotNull = [item[1] for item in columnInfos if item[3] == 1]

    df = pd.read_csv(arq,sep="|",
                     encoding="utf16",header=None,names=colunas,lineterminator ='\n')
    for col in columnsNotNull:
        df.dropna(subset=[col], inplace=True)
        
    df = df.replace(r'\r','', regex=True) 
    df_obj = df.select_dtypes('object')
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
    df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

app_log(1,"Tabelas Fato")
for arq in filter(lambda file: "Fact" in file,dir_list):
    cursor = conn.execute("select * from {}".format(arq[5:-4]))
    colunas = [description[0] for description in cursor.description]
    columnsQuery = cursor.execute(f"pragma table_info('{arq[5:-4]}')")
    columnInfos = cursor.fetchall()
    columnsNotNull = [item[1] for item in columnInfos if item[3] == 1]

    df = pd.read_csv(arq,sep="|",
                     encoding="utf16",header=None,names=colunas,lineterminator ='\n')
    for col in columnsNotNull:
        df.dropna(subset=[col], inplace=True)
    df = df.replace(r'\r','', regex=True) 
    df_obj = df.select_dtypes('object')
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

    app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
    df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

app_log(1,"Demais Tabelas ")
for arq in filter(lambda file: "Fact" not in file and "Dim" not in file ,dir_list):
    try:
        cursor = conn.execute("select * from {}".format(arq[5:-4]))
        colunas = [description[0] for description in cursor.description]
        columnsQuery = cursor.execute(f"pragma table_info('{arq[5:-4]}')")
        columnInfos = cursor.fetchall()
        columnsNotNull = [item[1] for item in columnInfos if item[3] == 1]

        df = pd.read_csv(arq,sep="|",
                        encoding="utf16",header=None,names=colunas,lineterminator ='\n')
        for col in columnsNotNull:
            df.dropna(subset=[col], inplace=True)
        df = df.replace(r'\r','', regex=True) 
        df_obj = df.select_dtypes('object')
        df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
        df = df.replace(r'\r','', regex=True) 
        df_obj = df.select_dtypes('object')
        df = df.apply(lambda x: x.str.strip(),axis=1)
        app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
        df.to_sql(arq[5:-4],conn,if_exists='append',index=False)
    except:
        df = pd.read_csv(arq,sep="|",
                        encoding="utf16",header=None,lineterminator ='\n',
                        keep_default_na=False)
        df = df.replace(r'\r','', regex=True) 
        df_obj = df.select_dtypes('object')
        df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
        app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
        df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

conn.close()

[22/01/2024 23:48:42] INFO - Carga de dados no banco de dados
 [22/01/2024 23:48:42] INFO - Tabelas Dimensao 
  [22/01/2024 23:48:42] INFO - Carregando DimAccount - 99 linhas
  [22/01/2024 23:48:42] INFO - Carregando DimCurrency - 105 linhas
  [22/01/2024 23:48:43] INFO - Carregando DimCustomer - 18484 linhas
  [22/01/2024 23:48:43] INFO - Carregando DimDate - 3652 linhas
  [22/01/2024 23:48:43] INFO - Carregando DimDepartmentGroup - 7 linhas
  [22/01/2024 23:48:43] INFO - Carregando DimEmployee - 296 linhas
  [22/01/2024 23:48:43] INFO - Carregando DimGeography - 655 linhas
  [22/01/2024 23:48:44] INFO - Carregando DimOrganization - 14 linhas
  [22/01/2024 23:48:44] INFO - Carregando DimProduct - 277 linhas
  [22/01/2024 23:48:44] INFO - Carregando DimProductCategory - 4 linhas
  [22/01/2024 23:48:44] INFO - Carregando DimProductSubcategory - 37 linhas
  [22/01/2024 23:48:44] INFO - Carregando DimPromotion - 16 linhas
  [22/01/2024 23:48:44] INFO - Carregando DimReseller - 701 linhas


In [5]:
conn = sqlite3.connect("database\db\dbo.db")
sql_query = """SELECT distinct Class FROM DimProduct;"""
df = pd.read_sql(sql_query,conn)
conn.close()
df

,Class
0,None
1,H
2,L
3,M


In [6]:
a = df.Class.str.strip()

In [7]:
a.values

array([None, 'H', 'L', 'M'], dtype=object)